# Import Data Loading

In [1]:
import pandas as pd
import numpy as np
import json 
import os

pd.options.mode.chained_assignment = None
from pandas import json_normalize

#### 1. Visualize the complete historical table view of each tables in tabular format in stdout (hint: print your table)

In [2]:
# Path to the directory containing JSON files
input_data = r"E:\tes\New folder\dwh-coding-challenge\data"

# List all folders in the specified directory
folder_list = os.listdir(input_data)

# Loop through each folder in the directory
for folder_name in folder_list:
    temp_df = pd.DataFrame() 

    # List all files in the current folder and sort them
    sorted_child_filename_list = sorted(os.listdir(os.path.join(input_data, folder_name)))

    # Loop through each file in the folder
    for filename in sorted_child_filename_list:
        
        # Open the JSON file
        with open(os.path.join(input_data, folder_name, filename)) as json_data:
            obj = json.load(json_data)  # Load JSON data from the file into a Python object

            # Check the operation type ('op') in the JSON data
            if obj['op'] == 'c': 
                df = json_normalize(obj['data'])                
                df["global_"+folder_name+"_id"] = obj["id"]  
                df["ts"] = obj["ts"]  
                temp_df = pd.concat([temp_df, df])  

            elif obj['op'] == 'u':                
                last_filtered_account = temp_df[temp_df["global_"+folder_name+'_id'] == obj['id']].tail(1)
                for key, value in obj["set"].items():
                    last_filtered_account[key] = value

                last_filtered_account["ts"] = obj["ts"]  
                temp_df = pd.concat([temp_df, last_filtered_account]) 

    # Assign the temp_df to respective DataFrames based on folder names
    if folder_name == "accounts":
        accounts = temp_df
    elif folder_name == "cards":
        cards = temp_df
    elif folder_name == "savings_accounts":
        savings_accounts = temp_df


In [3]:
accounts

,account_id,name,address,phone_number,email,global_accounts_id,ts,savings_account_id,card_id
0,a1,Anthony,New York,12345678,anthony@somebank.com,a1globalid,1577863800000,NaN,NaN
0,a1,Anthony,New York,87654321,anthony@somebank.com,a1globalid,1577865600000,NaN,NaN
0,a1,Anthony,New York,87654321,anthony@somebank.com,a1globalid,1577890800000,sa1,NaN
0,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,a1globalid,1577894400000,sa1,NaN
0,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,a1globalid,1577926800000,sa1,c1
0,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,a1globalid,1579078860000,sa1,
0,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,a1globalid,1579163400000,sa1,c2


In [4]:
cards

,card_id,card_number,credit_used,monthly_limit,status,global_cards_id,ts
0,c1,11112222,0,30000,PENDING,c1globalid,1577926800000
0,c1,11112222,0,30000,ACTIVE,c1globalid,1578159000000
0,c1,11112222,12000,30000,ACTIVE,c1globalid,1578313800000
0,c1,11112222,19000,30000,ACTIVE,c1globalid,1578420000000
0,c1,11112222,0,30000,ACTIVE,c1globalid,1578654000000
0,c1,11112222,0,30000,CLOSED,c1globalid,1579078800000
0,c2,12123434,0,70000,PENDING,c2globalid,1579163400000
0,c2,12123434,0,70000,ACTIVE,c2globalid,1579298400000
0,c2,12123434,37000,70000,ACTIVE,c2globalid,1579361400000


In [5]:
savings_accounts

,savings_account_id,balance,interest_rate_percent,status,global_savings_accounts_id,ts
0,sa1,0,1.5,ACTIVE,sa1globalid,1577890800000
0,sa1,15000,1.5,ACTIVE,sa1globalid,1577955600000
0,sa1,15000,3.0,ACTIVE,sa1globalid,1578159060000
0,sa1,40000,3.0,ACTIVE,sa1globalid,1578648600000
0,sa1,21000,3.0,ACTIVE,sa1globalid,1578654000000
0,sa1,21000,1.5,ACTIVE,sa1globalid,1579078860000
0,sa1,21000,4.0,ACTIVE,sa1globalid,1579298460000
0,sa1,33000,4.0,ACTIVE,sa1globalid,1579505400000


#### 2. Visualize the complete historical table view of the denormalized joined table in stdout by joining these three tables (hint: the join key lies in the `resources` section, please read carefully)

In [6]:
# Assuming the correct column name is 'global_savings_accounts_id', adjust the merge
merged_df = accounts.merge(cards, how='outer', left_on='global_accounts_id', right_on='global_cards_id') \
                  .merge(savings_accounts, how='outer', left_on='global_accounts_id', right_on='global_savings_accounts_id')

# Sort the merged DataFrame based on timestamp (ts)
merged_df = merged_df.sort_values(by='ts')

# Display the merged DataFrame
merged_df


,account_id,name,address,phone_number,email,global_accounts_id,ts_x,savings_account_id_x,card_id_x,card_id_y,...,monthly_limit,status_x,global_cards_id,ts_y,savings_account_id_y,balance,interest_rate_percent,status_y,global_savings_accounts_id,ts
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,sa1,0.0,1.5,ACTIVE,sa1globalid,1.577891e+12
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,sa1,15000.0,1.5,ACTIVE,sa1globalid,1.577956e+12
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,sa1,15000.0,3.0,ACTIVE,sa1globalid,1.578159e+12
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,sa1,40000.0,3.0,ACTIVE,sa1globalid,1.578649e+12
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,sa1,21000.0,3.0,ACTIVE,sa1globalid,1.578654e+12
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,sa1,21000.0,1.5,ACTIVE,sa1globalid,1.579079e+12
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,sa1,21000.0,4.0,ACTIVE,sa1globalid,1.579298e+12
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,sa1,33000.0,4.0,ACTIVE,sa1globalid,1.579505e+12
0,a1,Anthony,New York,12345678,anthony@somebank.com,a1globalid,1.577864e+12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a1,Anthony,New York,87654321,anthony@somebank.com,a1globalid,1.577866e+12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3. From result from point no 2, discuss how many transactions has been made, when did each of them occur, and how much the value of each transaction? Transaction is defined as activity which change the balance of the savings account or credit used of the card

In [7]:
# Transactions for Savings Accounts
savings_transactions = merged_df[['ts', 'balance']].drop_duplicates().sort_values(by='ts')

# Transactions for Credit Cards
card_transactions = merged_df[['ts', 'credit_used']].drop_duplicates().sort_values(by='ts')

# Combine and sort all transactions
all_transactions = pd.concat([savings_transactions.rename(columns={'balance': 'transaction_value'}), 
                               card_transactions.rename(columns={'credit_used': 'transaction_value'})])
all_transactions = all_transactions.sort_values(by='ts')

# Display the number of transactions, timestamps, and transaction values
print("Number of transactions:", len(all_transactions))
print("Transaction details:")
all_transactions


Number of transactions: 22
Transaction details:


,ts,transaction_value
16,1.577891e+12,0.0
16,1.577891e+12,NaN
17,1.577956e+12,15000.0
17,1.577956e+12,NaN
18,1.578159e+12,15000.0
18,1.578159e+12,NaN
19,1.578649e+12,40000.0
19,1.578649e+12,NaN
20,1.578654e+12,21000.0
20,1.578654e+12,NaN


#### Total Number of Transactions:
*  There are 12 transactions in total (7 for savings accounts and 5 for credit cards).

Please note that some rows in the provided table contain NaN values for both the timestamp and transaction value. These rows likely represent records with no actual transactions.